In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
# from shapely.geometry import Point, box, LineString
# from shapely import intersection, difference, touches, contains
# from tqdm.notebook import tqdm
# import os
# import glob

target_epsg = 6350  # https://epsg.org/crs_6350/NAD83-2011-Conus-Albers.html?sessionkey=nb47agmo4r
# target_epsg_verbose = "EPSG:6350"

google_epsg = 4326 # web mercator: https://en.wikipedia.org/wiki/Web_Mercator_projection; https://geopandas.org/en/stable/docs/user_guide/projections.html#re-projecting

In [28]:
fn_joined_tracts_and_coords = '/Volumes/Extreme SSD/largest_plots/clean_data/joined_best_tract_and_coords.parquet'

In [2]:
fn_coords = '/Volumes/Extreme SSD/largest_plots/clean_data/coords.parquet'
Coords = gpd.read_parquet(fn_coords).rename(columns={'geometry':'coord_geometry'}).set_geometry('coord_geometry')#.set_crs(epsg=target_epsg)

In [3]:
fn_best_tract_out = '/Volumes/Extreme SSD/largest_plots/clean_data/best_tract.parquet'
BestTracts = gpd.read_parquet(fn_best_tract_out)

In [15]:
AdjCoords = Coords[['ba_code', 'fos_id', 'latitude', 'longitude', 'plant_name_eia', 'street_address', 'state', 'zip_code', 'coord_geometry']]
AdjCoords = AdjCoords.to_crs(epsg=google_epsg)
AdjCoords.index.names = ['aoi_index']

In [16]:
AdjBestTracts = BestTracts.to_crs(epsg=google_epsg)
AdjBestTracts['area_km'] = AdjBestTracts['area_m'] / (1e6)
AdjBestTracts

best_tract_geometry  distance_m  \
aoi_index                                                                  
0          POLYGON ((-77.76615 38.55025, -77.76616 38.550...         0.0   
1          POLYGON ((-77.51557 38.06962, -77.51584 38.069...         0.0   
2          POLYGON ((-85.97501 33.57985, -85.97571 33.579...         0.0   
3          POLYGON ((-89.92549 31.59760, -89.92547 31.597...         0.0   
4          POLYGON ((-84.08143 37.89224, -84.08141 37.892...         0.0   
...                                                      ...         ...   
226        POLYGON ((-88.23659 41.99648, -88.23667 41.995...         0.0   
227        POLYGON ((-87.75865 41.42839, -87.75875 41.428...         0.0   
228        POLYGON ((-89.10280 42.25045, -89.10268 42.250...         0.0   
229        POLYGON ((-74.16647 40.05956, -74.16626 40.059...         0.0   
230        POLYGON ((-94.27469 36.18483, -94.27490 36.184...         0.0   

                 area_m  does_tract_contain_coord   area_km  
aoi_index                                                    
0          8.275338e+05                      True  0.827534  
1          2.159902e+05                      True  0.215990  
2          4.148185e+06                      True  4.148185  
3          8.109797e+06                      True  8.109797  
4          9.675407e+06                      True  9.675407  
...                 ...                       ...       ...  
226        8.972320e+05                      True  0.897232  
227        1.762155e+06                      True  1.762155  
228        4.941363e+05                      True  0.494136  
229        8.120913e+05                      True  0.812091  
230        2.205352e+06                      True  2.205352  

[231 rows x 5 columns]

In [17]:
# Sanity check-- size of tracts
AdjBestTracts['area_km'].quantile(np.arange(0, 1.1, 0.1))

0.0      0.004655
0.1      0.275124
0.2      0.649050
0.3      0.897232
0.4      1.235784
0.5      1.762155
0.6      2.110230
0.7      2.586126
0.8      3.921957
0.9      7.259349
1.0    145.681388
Name: area_km, dtype: float64

In [32]:
Joined = AdjCoords.join(AdjBestTracts)
Joined = Joined.melt(
    id_vars=['ba_code',	'fos_id', 'latitude', 'longitude', 'plant_name_eia', 'street_address', 'state', 'zip_code', 'distance_m', 'area_m', 'does_tract_contain_coord', 'area_km'],
    value_vars=['coord_geometry', 'best_tract_geometry'],
    var_name='geometry_type',
    value_name='geometry',
    ignore_index=False
    
)    
Joined.to_parquet(fn_joined_tracts_and_coords)